# Discussion 04

## PCA in Practice

Welcome to Discussion 04. In this discussion, we'll see how to use PCA in practice. This will be useful preparation for this week's homework.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.decomposition

plt.rcParams['figure.figsize'] = (7,7)

Let's start by downloading some data:

In [ ]:
%%bash
if [[ ! -e "mnist.npz" ]]; then
    curl 'https://f000.backblazeb2.com/file/jeldridge-data/mnist.npz' --output mnist.npz
fi

In [ ]:
X = np.load('mnist.npz')['train'].T.astype(float)
X.shape

**Question 01**. Compute the principal component using numpy and plot it as a 28 x 28 image.

In [ ]:
# BEGIN SOLUTION
C = np.cov(X.T)
eigvals, eigvecs = np.linalg.eigh(C)
u = eigvecs[:, -1]

plt.imshow(u.reshape((28, -1)))
# END SOLUTION

**Question 02**. Compute the principal component using `sklearn.decomposition.PCA()`. Plot it to verify.

Note: you may (or may not) see that the colors are inverted compared to those above. Why is this?

In [ ]:
# BEGIN SOLUTION
pca = sklearn.decomposition.PCA()
Z = pca.fit_transform(X)

plt.imshow(pca.components_[0].reshape((28, -1)))
# END SOLUTION

Notice that `sklearn` adopts the convention that the principal components are the *rows* of the matrix, not the columns. That's OK -- we just need to keep it in mind.

Performing PCA is just a few lines of code with either numpy or sklearn. Feel free to use either, but make sure to read the docs!

When performing dimensionality reduction with PCA, we have a choice of how many principal components, $k$, to use. This choice determines the eventual dimensionality of the transformed data. Choosing $k$ to be too small means losing most of the information in the data; if we choose it to be large, we've defeated the purpose of dimensionality reduction. So how do we choose $k$?

One approach is to make a "scree" plot and look for an elbow, similar to what we did with k-means. The next few questions will explore this method.

**Question 03**. PCA makes the assumption that "variance is interesting". Therefore, it chooses as the first principal component the vector in the direction of greatest variance.

Using `np.var`, compute the variance of the first PCA feature.

In [ ]:
np.var(Z[:, 0]) # SOLUTION

If we compute the variance of the second PCA feature, will it be smaller or larger? Why? Try it!

In [ ]:
np.var(Z[:, 1]) # SOLUTION

**Question 04**. As we move away from the top eigenvector, the principal components become less and less "interesting" -- that is, they point in directions with less variance. Make an array with 784 entries that contains the variance of each of the PCA features, and plot it as a line graph.

In [ ]:
plt.plot(np.var(Z, axis=0)) # SOLUTION

**Question 05.** Plot the eigenvalues associated with each of the principal components in order from largest to smallest. What do you notice?

In [ ]:
plt.plot(np.flip(eigvals)) # SOLUTION

This is perhaps not surprising -- the eigenvalue, after all, *is* the variance in the direction of the eigenvector.

**Question 06**. Compute the *total* variance of all PCA features.

In [ ]:
np.sum(eigvals) # SOLUTION

Now compute the *total* variance of each of the original features. That is, the variance of pixel 1, plus the variance of pixel 2, etc. What do you notice?

In [ ]:
np.sum(np.var(X, axis=0)) # SOLUTION

The total variance is conserved by PCA. PCA just "rotates" the data so that the direction of greatest variance in the data is aligned with the first standard basis vector, and so on. For that reason, the eigenvalue of a principal component is sometimes referred to as the "variance explained" by that component.

**Question 07**. Complete the function below so that it takes in a parameters `i` and `k` and computes the reconstruction of image `i` (that is, `X[i]`) using only the top `k` principal components.

In [ ]:
def compute_reconstruction(i, k):
    U = pca.components_[:k] # SOLUTION
    return (U @ X[i]) @ U # SOLUTION NO PROMPT

The below function will plot your reconstruction.

In [ ]:
def plot_reconstruction(i, k):
    fig, ax = plt.subplots(1, 2)
    z = compute_reconstruction(i, k)
    ax[0].imshow(X[i].reshape((28, -1)), origin='upper')
    ax[0].set_title('original')
    
    ax[1].imshow(z.reshape((28, -1)))
    ax[1].set_title('Reconstructed')

Let's try it for image 15,000, starting with only 5 principal components.

In [ ]:
plot_reconstruction(15_000, k=5)

**Question 08**. Adjust `k` so that the reconstructed image looks reasonably close to the original. It doesn't have to look *exactly* the same, but close enough that you can tell that they are the same image.

In [ ]:
plot_reconstruction(15_000, k=100) # SOLUTION

Compare this value of `k` with the location of the elbow in your plots above.

In [ ]:
plt.plot(np.flip(eigvals))
plt.ylabel('Variance Explained')
plt.xlabel('k')

Notice that the plot is relatively flat and close to zero after $k = 150` or 200. This means that the variance (a.k.a., "interestingness") added by including an additional component is marginal. This suggests that we need only include around 150 to 200 components and we will keep most of the variance (the interestingness).

In other words, our original data lies in 784 dimensions, yes, but in reality most of the data points are actually quite close to a 150 dimensional hyperplane spanned by the top 150 eigenvectors of the covariance matrix. We lose little information by just projecting the data onto this hyperplane, allowing us to represent each image with only 150 numbers, instead of the full 784.